# De-anonymization: Min, Max and Time

### Introduction

In previous notebooks, I showed the meaning of some tags: first [some time-aggregation tags](https://www.kaggle.com/gregorycalvez/de-anonymization-time-aggregation-tags) and then [price, quantity and partial identification of stocks](https://www.kaggle.com/gregorycalvez/de-anonymization-price-quantity-stocks). In this notebook, I continue this work. A significant part of the pre-processing code is in the (public) utility file. 


##### Pre-requisites
Features are defined by tags. 
- `tag_6` means a price features. 
- `tag_6` features and `feature_0` interact together, you need to multiply `tag_6` features with `feature_0` to get something "meaningful". 
-  I consider `feature_3` x `feature_0` as the (normalized) price of the asset at the time of the trade. 
- `tag_14` gives away the id of the stock, you can group trades by stock / day. I am yet to fully identify the stocks. 

##### What do we show here? 
Here we show that: 
- `tag_12` means minimum
- `tag_13` means maximum
- `tag_22` means time (or some kind of clock)

In [ ]:
# Import and preprocessing
import pandas as pd
import datatable as dt 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly_express as px
import warnings
warnings.filterwarnings('ignore')

from imp import reload
import janest_utility as utility

u = utility.Utility('/kaggle/input/jane-street-market-prediction/')

train_datatable = dt.fread(u.filepath_train())
train = train_datatable.to_pandas()
train = u.add_intraday_ts(train)
train = u.add_stock_id_all(train)
n_trades = u.get_n_trades(train)
train = u.add_feature_0(train)

print('Biggest Stock / Day number of trades')
display(n_trades.head(10))

# Select only one stock / day for plots
stock = train[(train['date'] == 380) & (train['stock_id'] == '10')]
del train  # Free RAM

### First plots


In the rest, I'll consider that `feature_3_x0` is the price (post-normalization) of the stock. Plotting `feature_3_x0` with time gives you a good idea of the intraday price chart of that asset. 


We consider here, only one stock. And we plot some price features as a pairpot. 

In [ ]:
display(u.select_tags('tag_6 & ~tag_9 & ~(tag_0 | tag_1 | tag_2 | tag_3 | tag_4)').astype(int))
features_to_study = [f'{f}_x0' for f in u.get_features('tag_6 & ~tag_9 & ~(tag_0 | tag_1 | tag_2 | tag_3 | tag_4)')]
sns.pairplot(stock[features_to_study])
plt.show()

`feature_37` and `feature_39` present interesting patterns, some "lines". So I clustered the points along those strips and tried to figure out what was their common characteristics. That's how I found what's next. 

### A bit more about `tag_12`
`feature_37` is `tag_6 & tag_12` so let's look into other features related to `tag_12`. It appears that `tag_12` is then related somehow to `tag_22` via `feature_{60, 61, 65, 66}`

Therefore, I looked into those 4 tags with a pairplot on the data. And we can see that `feature_60` and `feature_61` are basically the same thing. More interestingly, they seem to be cluster arount some values (actually piecewise constant, found out after the clustering). So I categorized the data with respect to `feature_62` below. With that plot, the meaning of `tag_12` and `tag_22` became obvious. 

In [ ]:
display(u.select_tags('tag_12 & ~(tag_0 | tag_1 | tag_2 | tag_3 | tag_4) & ~tag_9').astype(int))
sns.pairplot(stock[['feature_60', 'feature_61', 'feature_65', 'feature_66']])
plt.show()

In [ ]:
fig, axs = plt.subplots(4, 1, sharex=True, figsize=(20, 10))
for i, f in enumerate(stock['feature_60'].unique()): 
    axs[0].scatter(
        stock[stock['feature_60'] == f]['intraday_ts'], 
        stock[stock['feature_60'] == f]['feature_3_x0'], 
        marker='.' , 
        label=i
    )
    axs[0].set_ylabel('Price: feature_3_x0')
    axs[1].scatter(
        stock[stock['feature_60'] == f]['intraday_ts'], 
        stock[stock['feature_60'] == f]['feature_37_x0'], 
        marker='.' , 
        label=i
    )
    axs[1].set_ylabel('feature_37')
    axs[2].scatter(
        stock[stock['feature_60'] == f]['intraday_ts'], 
        stock[stock['feature_60'] == f]['feature_60'], 
        marker='.' , 
    )
    axs[2].set_ylabel('feature_60')
    axs[3].scatter(
        stock[stock['feature_60'] == f]['intraday_ts'], 
        stock[stock['feature_60'] == f]['feature_65'], 
        marker='.' , 
    )
    axs[3].set_ylabel('feature_65')
fig.legend()
plt.show()

### Minimum Tag 

With this plot, we can guess that `tag_12` represents a minimum. 


##### Intuitively
Here, I try to describe the different feature in plain english. I did not find difference between `feature_65` and `feature_66` and between `feature_60` and `feature_61`. So I will speak only about `feature_60` and `feature_65`. 

- `feature_37` is the current price minus the rolling minimum of the price during the day. So, whenever we reach a new minimum, we go back to 0. That's the reason why we had those strips on the pairplot. 
- `feature_60` is the time at which we reach that minimum. `feature_60` changes only when the price reaches a new minimum. Hence, it's piecewise constant.
- `feature_65` is the time since we reached the rolling min. So, like `feature_37`, it goes back to 0 whenever we reach a new minimum. And then it increases linearly with time. 

##### Formalized 
Let me formalize this a bit more. 
Let's assume that we have a price time series $p_t$. 

Let's define the time series $\hat p_t$ the rolling minimum, ie.: 
$$\hat p_t = \min_{s<t} p_s $$

Furthermore, we can count the number of times we reach a new minimum, let's call this number $n_t$. In the example above, we have 19 times during the day when the price reached a new minimum. \
**Remark 1:** That number can grow very quickly if the stock is plummeting, but also stay to 0 if the stock price is increasing during the whole day. \
**Remark 2:** We have always $n_{t=0} = 0$, at the begining of the day, we did not reach a minimum. And on the example above, $n_{t=1} = 19$. 

For each $n$, we call $T_n$ the time when we reach the new minimum and $\hat p_n$ that minimum.  
**Remark 3:** We have $\hat p_t = p_{\hat T_{n_t}}$

I say that we have: 
$$ \text{feature_37}_t = p_t - p_{\hat T_{n_t}} $$

$$ \text{feature_65/66}_t = t - T_{n_t} $$

$$ \text{feature_60/61}_t = T_{n_t} $$

##### A graph
Maybe the following graph will be helpful to show the relation between price, `feature_60` and `feature_37`. 

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(
    stock['feature_3_x0'], 
    stock['feature_37_x0'], 
    c=stock['feature_60']
)
plt.colorbar()
plt.xlabel('Price')
plt.ylabel('`feature_39`')
plt.title('Price and `feature_{39, 60}`, color is `feature_60`')
plt.show()

### Maximum Tag `tag_13`

Of course if there is a min, there is a max. Below I show similar plots on the same stock with `tag_13` which is the max tag. 

In [ ]:
display(u.select_tags('tag_13 & ~(tag_0 | tag_1 | tag_2 | tag_3 | tag_4) & ~tag_9').astype(int))

fig, axs = plt.subplots(4, 1, sharex=True, figsize=(20, 10))
values_62 = stock['feature_62'].unique().tolist()
np.random.shuffle(values_62)
for i, f in enumerate(values_62): 
    axs[0].scatter(
        stock[stock['feature_62'] == f]['intraday_ts'], 
        stock[stock['feature_62'] == f]['feature_3_x0'], 
        marker='.' , 
        label=i
    )
    axs[0].set_ylabel('Price: feature_3_x0')
    axs[1].scatter(
        stock[stock['feature_62'] == f]['intraday_ts'], 
        stock[stock['feature_62'] == f]['feature_39_x0'], 
        marker='.' , 
        label=i
    )
    axs[1].set_ylabel('feature_39')
    axs[2].scatter(
        stock[stock['feature_62'] == f]['intraday_ts'], 
        stock[stock['feature_62'] == f]['feature_62'], 
        marker='.' , 
    )
    axs[2].set_ylabel('feature_62')
    axs[3].scatter(
        stock[stock['feature_62'] == f]['intraday_ts'], 
        stock[stock['feature_62'] == f]['feature_67'], 
        marker='.' , 
    )
    axs[3].set_ylabel('feature_67')
fig.legend()
plt.show()

### Time Tag `tag_22`

That leaves us with `tag_22` being homogenous to a time. 

After this min / max discovery, I figured that some notebooks already looked into `tag_22` and specifically `feature_64` being a "tick clock" and I won't reproduce those graphs here. You can read [Carl's notebook](https://www.kaggle.com/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook) which covers many other things! I hope that this notebook completes their points about the role of time in the features. 

In [ ]:
display(u.select_tags('tag_22').astype(int))

# Conclusion

Let me know your thoughts on this notebook and the meaning of those tags. I am still working on other tags but wanted to share what I had already. 


### PS: On the importance of de-anonymization
In the previous notebooks, I received some negative comments as de-anonymization "will ruin the competition". I wanted to answer here. 


##### De-anonymization changes nothing (see [host's discussion](https://www.kaggle.com/c/jane-street-market-prediction/discussion/202697))
First, the host will only change the final test set. So in short, nothing will change apart the size of the test set or the duration of the scoring (adding few more months). \
Let's assume that someone is able to find which assets are in the data and identify them. The current final test set is planned to have data form 2020-08 to 2021-02. So competitors will be able to incorporate those returns in the models. \
If you're able to de-anonymize sufficiently, you can basically make perfect prediction on that period. 

##### De-anonymization is already done
I am just sharing things I have figured out by myself. And if I was able to figure them out, others might as well. \
Furthermore, I'd argue that people can already identify the market (e.g. NYSE stocks or Tokyo Stocks) if not all the stocks already. One could bias their models with the "index returns", gaining an edge on other competitors without making perfect predictions. 

##### Feature Engineering is important 
Feature Engineering is one of the most important things in data science. And we're here to learn things; you can't learn much on anonymized data. 

So if you want to learn something about the markets and understand why the host included those min / max features, see: [Investopia Support / Resistance](https://www.investopedia.com/trading/support-and-resistance-basics/)